In [3]:
from scripts.lasso_tool import lasso_tool

file_path = "/home/george-vengrovski/Documents/projects/song_analysis_pipeline/files/labels_HDBSCAN_Classification.npz"
output_file_path = "spectrogram_segments.png"

lasso_tool(file_path, output_file_path)

FileNotFoundError: [Errno 2] No such file or directory: 'path_to_your_data.npz'